## 이미지 데이터를 읽어와서 전처리 및 학습
- 전체 플로우
- 이미지 폴더를 라벨별로 나누는 작업을 먼저 해도 됨

In [6]:
import tensorflow as tf

import os
from glob import glob
from tensorflow.keras import layers
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(os.listdir('../dataset/mnist/training'))

train_dir = '../dataset/mnist/training/'
test_dir = '../dataset/mnist/testing/'

print(os.listdir(train_dir), '\n', os.listdir(test_dir))

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] 
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


## HyperParamter

In [12]:
num_epochs=10
batch_size=32

learning_rate = 0.001
dropout_rate = 0.7

input_shape = (28,28,1)
num_classes = 10

## ImageDataGenerator

In [8]:
# train용 , test 용 Image Data Generator 생성
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

# test 용 Data Generator
test_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [14]:
# train generator 에 대한 이미지 연결
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size = input_shape[:2], # (28,28)
    batch_size = batch_size, # 32
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

# test data generator 에 대한 이미지 연결
valid_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size = input_shape[:2],
    batch_size = batch_size,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


## Build Model

In [18]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3,3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size = (2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs = net, name = 'BasicCNN')
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])


In [24]:
# 학습진행
# train generator 로 생성한 것으로 학습하고, valid generator 로 생성한 것으로 테스트
model.fit_generator(
    train_generator, # train generator
    steps_per_epoch = len(train_generator), # 1875
    epochs = 1, # num_epochs, # 
    validation_data = valid_generator,
    validation_steps = len(valid_generator)
)

1875/1875 [==============================] - 666s 355ms/step - loss: 0.5596 - accuracy: 0.8131 - val_loss: 0.2069 - val_accuracy: 0.9368
